In [49]:
import {Complex,creal,cstr,cadd,cmul,cinv,cabs,cdiv} from './comp.js'
import {Polynomial,pstr,pdeg,peval,pdiff,pGCD} from './poly.js'

function abStep(poly,guess) {
    if(pdeg(poly)!=guess.length) {console.log("Guess wrong length"); return 0}
    var dp=pdiff(poly),dpz,pz,z,s;
    var update=[],i,j,ps,zs,os;
    for(i=0;i<guess.length;i++) {
        z = guess[i];
        pz = peval(poly,z);
        if(cabs(pz)<1e-10) {
            update.push(z);
        } else {
            ps = creal(0);
            for(j=0;j<guess.length;j++) {
                if(i!=j) {
                    s = guess[j];
                    zs = cadd(z,cmul(creal(-1),s));
                    if(cabs(zs)<1e-6) {console.log("Pole sum error: zeros too close"); return 0}
                    ps = cadd(ps,cinv(zs));
                }
            }
        dpz = peval(dp,z);
        os = cadd(cdiv(dpz,pz),cmul(creal(-1),ps));            
        update.push(cadd(z,cmul(creal(-1),cinv(os))));
        }
    }
    return update
}

function abRecur(poly,guess) {
    var update,i,change=0;
    update = abStep(poly,guess);
    if(update==0) {console.log("Error"); return}
    for(i=0;i<guess.length;i++) {
        change += cabs(cadd(update[i],cmul(creal(-1),guess[i])))
    }
    if (change<1e-10) {return update}
    return abRecur(poly,update)
}

function zBound(poly) {
    return Math.max(...(poly.coeff.slice(0,-1).map((v)=>cabs(v))))/cabs(poly.coeff.slice(-1)[0])+1
}

function zGuess(poly) {
    var deg=pdeg(poly), guess=[], i;
    var rb=zBound(poly),r,angle;
    for(i=0;i<deg;i++) {
        r = rb*Math.sqrt(Math.random());// even area coverage
        angle=2*Math.PI*Math.random();
        guess[i] = new Complex([r*Math.sin(angle),r*Math.cos(angle)]);
    }
    return guess
}

function abwGuess(poly) {
    return abRecur(poly,zGuess(poly))
}

In [120]:
var testp = new Polynomial([creal(6),creal(5),creal(1)])

var testdp = pdiff(testp)

console.log("Test polynomial for multiple roots: \n",pGCD(testp,testdp));

console.log("\nTest polynomial root step: \n",abStep(testp,[new Complex([2,1]),new Complex([4,-5])]));

var twop = new Polynomial([creal(-1.5),creal(-0.25),creal(1)]);

console.log("\nTwo-day zeros: \n", abRecur(twop,[new Complex([2,1]),new Complex([4,-5])]));

var threep = new Polynomial([creal(-1),creal(-0.75),creal(-0.25),creal(1)]);

console.log("\nThree-day zeros: \n",abRecur(threep,[new Complex([2,1]), new Complex([4,-5]), new Complex([0.25,-0.75])]));

var multip = new Polynomial([creal(1),creal(2),creal(1)]);

console.log("\nMultiple zeros: \n", abRecur(multip,[new Complex([2,1]),new Complex([4,-5])]));

console.log("\nRandom guess: \n", abwGuess(multip));

Test polynomial for multiple roots: 
 Polynomial { coeff: [ Complex { re: -0.25, im: 0 } ] }

Test polynomial root step: 
 [ Complex { re: -0.06859382655561008, im: 1.2283194512493876 },
  Complex { re: -2.995121951219513, im: -5.022764227642275 } ]

Two-day zeros: 
 [ Complex { re: 1.356107225224484, im: 8.517756541325666e-15 },
  Complex { re: -1.1061072252244297, im: -6.484267604193322e-14 } ]

Three-day zeros: 
 [ Complex { re: 1.3519136862464185, im: -1.1871094207397087e-13 },
  Complex { re: -0.5509568431238944, im: 0.6604080182084614 },
  Complex { re: -0.5509568431231111, im: -0.6604080182084816 } ]

Multiple zeros: 
 [ Complex { re: -0.9999968638743151, im: 0.0000031361251247801898 },
  Complex { re: -1.000003136125326, im: -0.0000031361333518193908 } ]

Random guess: 
 [ Complex { re: -0.9999962126658939, im: 6.638300852262926e-7 },
  Complex { re: -1.00000378733774, im: -6.638111263403167e-7 } ]
